## Libraries

In [1]:
try:
    import google.colab
    from google.colab import drive
    drive.mount('/content/drive', force_remount = True)
    IN_COLAB = True
except:
    IN_COLAB = False

Mounted at /content/drive


In [2]:
if IN_COLAB:
    !pip install transformers
    !pip install datasets
    !pip install evaluate
    !pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 5.0 MB/s 
     |████████████████████████████████| 7.6 MB 73.9 MB/s 
     |████████████████████████████████| 182 kB 79.8 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 452 kB 5.1 MB/s 
     |████████████████████████████████| 132 kB 92.3 MB/s 
     |████████████████████████████████| 212 kB 82.8 MB/s 
     |████████████████████████████████| 127 kB 92.0 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 81 kB 4.2 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/

In [3]:
import os
import torch

if IN_COLAB:
    root_path = '/content/drive/MyDrive/Knowledge/MSIT/Research/InstructABSA'
else:
    root_path = '/Users/kscaria/Library/CloudStorage/GoogleDrive-scariakevin1@gmail.com/My Drive/Knowledge/MSIT/Research/InstructABSA'
    
use_mps = True if torch.has_mps else False
os.chdir(root_path)

In [4]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
from sklearn.metrics import f1_score, confusion_matrix

from InstructABSA.utils import T5ATSC
from InstructABSA.data_prep import ModelReadyData

In [26]:
rest_train_file_path = './Data/semeval14/ABSA_TrainData/Restaurants_Train_v2.csv'
laptops_train_file_path = './Data/semeval14/ABSA_TrainData/Laptop_Train_v2.csv'
rest_test_file_path = './Data/semeval14/ABSA_Gold_TestData/Restaurants_Test_Gold.csv'
laptops_test_file_path = './Data/semeval14/ABSA_Gold_TestData/Laptops_Test_Gold.csv'
rest_val_file_path = './Data/semeval14/ABSA_TrainData/Rest_val.csv'
laptops_val_file_path = './Data/semeval14/ABSA_TrainData/Lap_val.csv'

# Load the data
res_tr_df = pd.read_csv(rest_train_file_path)
lap_tr_df = pd.read_csv(laptops_train_file_path)
res_te_df = pd.read_csv(rest_test_file_path)
lap_te_df = pd.read_csv(laptops_test_file_path)
res_val_df = pd.read_csv(rest_val_file_path)
lap_val_df = pd.read_csv(laptops_val_file_path)

lap_tr_df =pd.concat([lap_tr_df, lap_val_df]).reset_index(drop = True)
res_tr_df =pd.concat([res_tr_df, res_val_df]).reset_index(drop = True)

In [27]:
# Extract the aspect term and polarity for each review
model_data = ModelReadyData()
res_tr_df = model_data.extract_rowwise_aspect_polarity(res_tr_df, on='aspectTerms', by="term", min_val=1)
lap_tr_df = model_data.extract_rowwise_aspect_polarity(lap_tr_df, on='aspectTerms', by="term", min_val=1)
res_te_df = model_data.extract_rowwise_aspect_polarity(res_te_df, on='aspectTerms', by="term", min_val=1)
lap_te_df = model_data.extract_rowwise_aspect_polarity(lap_te_df, on='aspectTerms', by="term", min_val=1)

# For aspects which are classified as noaspectterm, the sentiment is none.

# Get the input text into the required format
bos_instruction_lap = """Definition: The output will be 'positive' if the aspect identified in the sentence contains a positive sentiment. If the sentiment of the identified aspect in the input is negative the answer will be 'negative'. 
For aspects with no sentiment in the input, the output should be 'neutral'.
Positive example 1-
input: I charge it at night and skip taking the cord with me because of the good battery life. The sentiment of aspect battery life is 
output: positive
Positive example 2-
input: I even got my teenage son one, because of the features that it offers, like, iChat, Photobooth, garage band and more!. The aspect is garage band.
output: positive
Negative example 1-
input: Speaking of the browser, it too has problems. The sentiment of aspect browser is
output: negative
Negative example 2-
input: The keyboard is too slick. The sentiment of aspect keyboard is
output: negative
Neutral example 1-
input: I took it back for an Asus and same thing- blue screen which required me to remove the battery to reset. The sentiment of aspect battery is
output: neutral
Neutral example 2-
input: Nightly my computer defrags itself and runs a virus scan. The sentiment of aspect virus scan is 
output: neutral
Now complete the following example-
input: """

bos_instruction_res = """Definition: The output will be 'positive' if the aspect identified in the sentence contains a positive sentiment. If the sentiment of the identified aspect in the input is negative the answer will be 'negative'. 
For aspects with no sentiment in the input, the output should be 'neutral'.
Positive example 1-
input: With the great variety on the menu , I eat here often and never get bored. The sentiment of aspect menu is
output: positive
Positive example 2- 
input: Great food, good size menu, great service and an unpretensious setting. The sentiment of aspect food is
output: positive
Negative example 1-
input: They did not have mayonnaise, forgot our toast, left out ingredients (ie cheese in an omelet), below hot temperatures and the bacon was so over cooked it crumbled on the plate when you touched it. The sentiment of aspect toast is
output: negative
Negative example 2-
input: The seats are uncomfortable if you are sitting against the wall on wooden benches. The sentiment of aspect seats is
output: negative
Neutral example 1-
input: I asked for seltzer with lime, no ice. The sentient of aspect seltzer with lime is
output: neutral
Neutral example 2-
input: They wouldnt even let me finish my glass of wine before offering another. The sentiment of aspect glass of wine is
output: neutral
Now complete the following example-
input: """
delim_instruction = ' The sentiment of aspect '
eos_insrtuction = ' is\noutput:'

res_tr_df = model_data.create_data_in_atsc_format(res_tr_df, 'raw_text', 'aspect', 'labels', bos_instruction_res, delim_instruction, eos_insrtuction)
lap_tr_df = model_data.create_data_in_atsc_format(lap_tr_df, 'raw_text', 'aspect', 'labels', bos_instruction_lap, delim_instruction, eos_insrtuction)
res_te_df = model_data.create_data_in_atsc_format(res_te_df, 'raw_text', 'aspect', 'labels', bos_instruction_res, delim_instruction, eos_insrtuction)
lap_te_df = model_data.create_data_in_atsc_format(lap_te_df, 'raw_text', 'aspect', 'labels', bos_instruction_lap, delim_instruction, eos_insrtuction)

In [28]:
# # Extract the aspect term and polarity for each review
# model_data = ModelReadyData()
# res_tr_df = model_data.extract_rowwise_aspect_polarity(res_tr_df, on='aspectTerms', by="term", min_val=1)
# lap_tr_df = model_data.extract_rowwise_aspect_polarity(lap_tr_df, on='aspectTerms', by="term", min_val=1)
# res_te_df = model_data.extract_rowwise_aspect_polarity(res_te_df, on='aspectTerms', by="term", min_val=1)
# lap_te_df = model_data.extract_rowwise_aspect_polarity(lap_te_df, on='aspectTerms', by="term", min_val=1)

# # For aspects which are classified as noaspectterm, the sentiment is none.

# # Get the input text into the required format
# bos_instruction_lap = """Definition: The output will be 'positive' if the aspect identified in the sentence contains a positive sentiment. If the sentiment of the identified aspect in the input is negative the answer will be 'negative'. 
# Otherwise, the output should be 'neutral'.
# Positive example 1-
# input: I charge it at night and skip taking the cord with me because of the good battery life. The aspect is battery life.
# output: positive
# Positive example 2-
# input: I even got my teenage son one, because of the features that it offers, like, iChat, Photobooth, garage band and more!. The aspect is garage band.
# output: positive
# Negative example 1-
# input: Speaking of the browser, it too has problems. The aspect is browser.
# output: negative
# Negative example 2-
# input: The keyboard is too slick. The aspect is keyboard.
# output: negative
# Neutral example 1-
# input: I took it back for an Asus and same thing- blue screen which required me to remove the battery to reset. The aspect is battery.
# output: neutral
# Neutral example 2-
# input: Nightly my computer defrags itself and runs a virus scan. The aspect is virus scan.
# output: neutral
# Now complete the following example-
# input: """

# bos_instruction_res = """Definition: The output will be 'positive' if the aspect identified in the sentence contains a positive sentiment. If the sentiment of the identified aspect in the input is negative the answer will be 'negative'. 
# Otherwise, the output should be 'neutral'.
# Positive example 1-
# input: With the great variety on the menu , I eat here often and never get bored. The aspect is menu.
# output: positive
# Positive example 2- 
# input: Great food, good size menu, great service and an unpretensious setting. The aspect is food.
# output: positive
# Negative example 1-
# input: They did not have mayonnaise, forgot our toast, left out ingredients (ie cheese in an omelet), below hot temperatures and the bacon was so over cooked it crumbled on the plate when you touched it. The aspect is toast.
# output: negative
# Negative example 2-
# input: The seats are uncomfortable if you are sitting against the wall on wooden benches. The aspect is seats.
# output: negative
# Neutral example 1-
# input: I asked for seltzer with lime, no ice. The aspect is seltzer with lime.
# output: neutral
# Neutral example 2-
# input: They wouldnt even let me finish my glass of wine before offering another. The aspect is glass of wine.
# output: neutral
# Now complete the following example-
# input: """
# delim_instruction = ' The aspect is '
# eos_insrtuction = '.\noutput:'

# res_tr_df = model_data.create_data_in_atsc_format(res_tr_df, 'raw_text', 'aspect', 'labels', bos_instruction_res, delim_instruction, eos_insrtuction)
# lap_tr_df = model_data.create_data_in_atsc_format(lap_tr_df, 'raw_text', 'aspect', 'labels', bos_instruction_lap, delim_instruction, eos_insrtuction)
# res_te_df = model_data.create_data_in_atsc_format(res_te_df, 'raw_text', 'aspect', 'labels', bos_instruction_res, delim_instruction, eos_insrtuction)
# lap_te_df = model_data.create_data_in_atsc_format(lap_te_df, 'raw_text', 'aspect', 'labels', bos_instruction_lap, delim_instruction, eos_insrtuction)

In [29]:
print(res_tr_df.iloc[0]['text'], res_tr_df.iloc[0]['labels'])

Definition: The output will be 'positive' if the aspect identified in the sentence contains a positive sentiment. If the sentiment of the identified aspect in the input is negative the answer will be 'negative'. 
For aspects with no sentiment in the input, the output should be 'neutral'.
Positive example 1-
input: With the great variety on the menu , I eat here often and never get bored. The sentiment of aspect menu is
output: positive
Positive example 2- 
input: Great food, good size menu, great service and an unpretensious setting. The sentiment of aspect food is
output: positive
Negative example 1-
input: They did not have mayonnaise, forgot our toast, left out ingredients (ie cheese in an omelet), below hot temperatures and the bacon was so over cooked it crumbled on the plate when you touched it. The sentiment of aspect toast is
output: negative
Negative example 2-
input: The seats are uncomfortable if you are sitting against the wall on wooden benches. The sentiment of aspect sea

In [30]:
# Filter only pos, neg and neut labels
res_tr_df = res_tr_df[res_tr_df['labels'].isin(['positive', 'negative', 'neutral'])]
res_te_df = res_te_df[res_te_df['labels'].isin(['positive', 'negative', 'neutral'])]
lap_tr_df = lap_tr_df[lap_tr_df['labels'].isin(['positive', 'negative', 'neutral'])]
lap_te_df = lap_te_df[lap_te_df['labels'].isin(['positive', 'negative', 'neutral'])]

In [31]:
# Resample to account for class imbalance
pos_res_tr_df = res_tr_df[res_tr_df['labels'] == 'positive'].sample(800, random_state=1999)
res_tr_df = pd.concat([pos_res_tr_df, res_tr_df[res_tr_df['labels'].isin(['negative', 'neutral'])]]).reset_index(drop=True)

In [32]:
res_tr_df['labels'].value_counts()

negative    823
positive    800
neutral     643
Name: labels, dtype: int64

In [33]:
# Experimentation
experiment_id = 'restaurants'
experiment_name = 'restaurants_instruct_pos_neg_neut'
model_checkpoint = 'allenai/tk-instruct-base-def-pos'

# Create T5 utils object
t5_exp = T5ATSC(model_checkpoint, experiment_id, res_tr_df, res_te_df, lap_tr_df, lap_te_df)

if IN_COLAB:
    model_out_path = os.path.join(root_path, 'T5', 'ATSC')
else:
    model_out_path = os.getcwd()

model_out_path = os.path.join(model_out_path, f"{model_checkpoint}-{experiment_name}", "checkpoints")
print('Model output path: ', model_out_path)


# Tokenize Datasets
id_dataset, ood_dataset, id_tokenized_dataset, ood_tokenized_dataset = t5_exp.set_data_for_training_semeval(experiment_id)

# Training arguments
training_args = {
    'output_dir':model_out_path,
    'evaluation_strategy':"epoch",
    'learning_rate':5e-5,
    'per_device_train_batch_size':8,
    'per_device_eval_batch_size':8,
    'num_train_epochs':4,
    'weight_decay':0.01,
    'warmup_ratio':0.1,
    'save_strategy':'no',
    'load_best_model_at_end':False,
    'push_to_hub':False,
    'eval_accumulation_steps':1,
    'use_mps_device':use_mps
}

Downloading:   0%|          | 0.00/2.13k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/677 [00:00<?, ?B/s]

...

Model output path:  /content/drive/MyDrive/Knowledge/MSIT/Research/InstructABSA/T5/ATSC/allenai/tk-instruct-base-def-pos-restaurants_instruct_pos_neg_neut/checkpoints


  0%|          | 0/3 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/3 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [34]:
# Train model
model_trainer = t5_exp.train(id_tokenized_dataset, **training_args)

***** Running training *****
  Num examples = 2266
  Num Epochs = 4
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 1136
  Number of trainable parameters = 247534848


Trainer device: cuda:0

Model training started ....


You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
1,No log,0.235203
2,0.401600,0.203366
3,0.401600,0.190175
4,0.255900,0.203481


***** Running Evaluation *****
  Num examples = 1120
  Batch size = 8
***** Running Evaluation *****
  Num examples = 1120
  Batch size = 8
***** Running Evaluation *****
  Num examples = 1120
  Batch size = 8
***** Running Evaluation *****
  Num examples = 1120
  Batch size = 8


Training completed. Do not forget to share your model on huggingface.co/models =)


Saving model checkpoint to /content/drive/MyDrive/Knowledge/MSIT/Research/InstructABSA/T5/ATSC/allenai/tk-instruct-base-def-pos-restaurants_instruct_pos_neg_neut/checkpoints
Configuration saved in /content/drive/MyDrive/Knowledge/MSIT/Research/InstructABSA/T5/ATSC/allenai/tk-instruct-base-def-pos-restaurants_instruct_pos_neg_neut/checkpoints/config.json
Model weights saved in /content/drive/MyDrive/Knowledge/MSIT/Research/InstructABSA/T5/ATSC/allenai/tk-instruct-base-def-pos-restaurants_instruct_pos_neg_neut/checkpoints/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/Knowledge/MSIT/Research/InstructABSA

In [35]:
# Model inference
best_model = 'checkpoints'
print('Getting model from path: ', model_out_path)

try:
    model_trainer
except:
    model_trainer = None

# Get prediction labels - Training set
id_tr_pred_labels, id_tr_preds = t5_exp.get_labels(predictor = model_trainer, tokenized_dataset = id_tokenized_dataset, sample_set = 'train', trained_model_path = model_out_path)

# Get prediction labels - Testing set
id_te_pred_labels, id_te_preds = t5_exp.get_labels(predictor = model_trainer, tokenized_dataset = id_tokenized_dataset, sample_set = 'validation', trained_model_path = model_out_path)

# Get prediction labels - OOD Training set
ood_tr_pred_labels, ood_tr_preds = t5_exp.get_labels(predictor = model_trainer, tokenized_dataset = ood_tokenized_dataset, sample_set = 'train', trained_model_path = model_out_path)

# Get prediction labels - OOD Testing set
ood_te_pred_labels, ood_te_preds = t5_exp.get_labels(predictor = model_trainer, tokenized_dataset = ood_tokenized_dataset, sample_set = 'validation', trained_model_path = model_out_path)

***** Running Prediction *****
  Num examples = 2266
  Batch size = 8


Getting model from path:  /content/drive/MyDrive/Knowledge/MSIT/Research/InstructABSA/T5/ATSC/allenai/tk-instruct-base-def-pos-restaurants_instruct_pos_neg_neut/checkpoints


***** Running Prediction *****
  Num examples = 1120
  Batch size = 8


***** Running Prediction *****
  Num examples = 2362
  Batch size = 8


***** Running Prediction *****
  Num examples = 638
  Batch size = 8


In [36]:
# Add new column in the respective dataframes
id_tr_df = pd.DataFrame(id_dataset['train'])
id_te_df = pd.DataFrame(id_dataset['validation'])
ood_tr_df = pd.DataFrame(ood_dataset['train'])
ood_te_df = pd.DataFrame(ood_dataset['validation'])

id_tr_df['pred_labels'] = id_tr_pred_labels
id_te_df['pred_labels'] = id_te_pred_labels
ood_tr_df['pred_labels'] = ood_tr_pred_labels
ood_te_df['pred_labels'] = ood_te_pred_labels

# Metrics
print('In domain train accuracy: ', id_tr_df[['labels', 'pred_labels']].apply(lambda x: x[0] == x[1], axis=1).sum()*100/len(id_tr_df))
print('In domain test accuracy: ', id_te_df[['labels', 'pred_labels']].apply(lambda x: x[0] == x[1], axis=1).sum()*100/len(id_te_df))

ood_df = pd.concat([ood_tr_df, ood_te_df])
print('Out of domain accuracy: ', ood_df[['labels', 'pred_labels']].apply(lambda x: x[0] == x[1], axis=1).sum()*100/len(ood_df))
print('Out of domain train accuracy: ', ood_tr_df[['labels', 'pred_labels']].apply(lambda x: x[0] == x[1], axis=1).sum()*100/len(ood_tr_df))
print('Out of domain test accuracy: ', ood_te_df[['labels', 'pred_labels']].apply(lambda x: x[0] == x[1], axis=1).sum()*100/len(ood_te_df))

#Dump outputs
dump_path = '/'.join(model_out_path.split('/')[:-1])
id_tr_filename = t5_exp.get_csv_filename(experiment_id)[0]
id_tr_df.to_csv(os.path.join(dump_path, 'posnegneuttk_' + id_tr_filename), index = False)

id_te_filename = t5_exp.get_csv_filename(experiment_id)[1]
id_te_df.to_csv(os.path.join(dump_path, 'posnegneuttk_' + id_te_filename), index = False)

ood_tr_filename = t5_exp.get_csv_filename(experiment_id)[2]
ood_tr_df.to_csv(os.path.join(dump_path, 'posnegneuttk_' + ood_tr_filename), index = False)

ood_te_filename = t5_exp.get_csv_filename(experiment_id)[3]
ood_te_df.to_csv(os.path.join(dump_path, 'posnegneuttk_' + ood_te_filename), index = False)

In domain train accuracy:  88.3053839364519
In domain test accuracy:  86.60714285714286
Out of domain accuracy:  79.96666666666667
Out of domain train accuracy:  79.2125317527519
Out of domain test accuracy:  82.75862068965517


In [39]:
print('Rest F1 Score: ', f1_score(id_te_df['labels'], id_te_df['pred_labels'], average = 'weighted'))
print('Lap F1 Score: ', f1_score(ood_te_df['labels'], ood_te_df['pred_labels'], average = 'weighted'))

Rest F1 Score:  0.8634959175530621
Lap F1 Score:  0.8254820515361349


In [37]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(id_te_df['labels'], id_te_df['pred_labels']))

[[172  17   7]
 [ 30 119  47]
 [ 24  25 679]]


## Upsampling Approach

In [ ]:
from transformers import T5Model, T5Tokenizer, T5ForConditionalGeneration, T5EncoderModel
from tqdm.notebook import tqdm
import numpy as np

In [ ]:
model = T5EncoderModel.from_pretrained('allenai/tk-instruct-base-def-pos')
tok = T5Tokenizer.from_pretrained('allenai/tk-instruct-base-def-pos')

Downloading:   0%|          | 0.00/677 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/495M [00:00<?, ?B/s]

Some weights of the model checkpoint at allenai/tk-instruct-base-def-pos were not used when initializing T5EncoderModel: ['decoder.block.11.layer.1.EncDecAttention.v.weight', 'decoder.block.1.layer.1.EncDecAttention.k.weight', 'decoder.block.2.layer.0.SelfAttention.o.weight', 'decoder.block.1.layer.0.layer_norm.weight', 'decoder.block.1.layer.1.EncDecAttention.q.weight', 'decoder.block.3.layer.0.SelfAttention.v.weight', 'decoder.block.2.layer.0.SelfAttention.v.weight', 'decoder.block.3.layer.1.EncDecAttention.v.weight', 'decoder.block.11.layer.0.SelfAttention.v.weight', 'decoder.block.7.layer.1.layer_norm.weight', 'decoder.block.11.layer.0.SelfAttention.k.weight', 'decoder.block.2.layer.2.DenseReluDense.wi_0.weight', 'decoder.block.5.layer.0.SelfAttention.v.weight', 'decoder.block.2.layer.1.EncDecAttention.o.weight', 'decoder.block.5.layer.2.DenseReluDense.wo.weight', 'decoder.block.7.layer.0.SelfAttention.o.weight', 'decoder.block.2.layer.2.DenseReluDense.wi_1.weight', 'decoder.block.

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.13k [00:00<?, ?B/s]

In [ ]:
def get_embeddings(dataframe, col_to_embed, model, tokenizer):
    emb = np.empty((dataframe.shape[0], 768))
    for idx, txt in tqdm(enumerate(dataframe[col_to_embed].tolist()[:])):
        input_ids = tokenizer(txt, return_tensors="pt").input_ids
        emb[idx] = torch.mean(model(input_ids=input_ids).last_hidden_state, dim=1).detach().numpy()
    return emb

In [ ]:
dump_path = './T5/ATSC/allenai/tk-instruct-base-def-pos-restaurants_instruct_pos_neg_neut'

In [ ]:
id_tr_emb = get_embeddings(res_tr_df, 'text', model, tok)
np.save(os.path.join(dump_path, 'res_tr_emb.npy'), id_tr_emb)

0it [00:00, ?it/s]

In [ ]:
id_te_emb = get_embeddings(res_te_df, 'text', model, tok)
np.save(os.path.join(dump_path, 'res_te_emb.npy'), id_te_emb)

0it [00:00, ?it/s]

In [ ]:
ood_tr_emb = get_embeddings(lap_tr_df, 'text', model, tok)
np.save(os.path.join(dump_path, 'lap_tr_emb.npy'), ood_tr_emb)

0it [00:00, ?it/s]

In [ ]:
ood_te_emb = get_embeddings(lap_te_df, 'text', model, tok)
np.save(os.path.join(dump_path, 'lap_te_emb.npy'), ood_te_emb)

0it [00:00, ?it/s]

In [ ]:
import tensorflow as tf
from tensorflow import keras
from imblearn.over_sampling import SMOTE

In [ ]:
sm = SMOTE(random_state = 42)
X_res, y_res = sm.fit_resample(id_tr_emb, res_tr_df['labels'].tolist())
y_res = pd.get_dummies(pd.DataFrame(y_res, columns = ['label'])).to_numpy()

In [ ]:
y_tr = pd.get_dummies(pd.DataFrame(res_tr_df['labels'].tolist(), columns = ['label'])).to_numpy()

In [ ]:
ann = tf.keras.models.Sequential()
ann.add(tf.keras.layers.Dense(units=768,activation="relu"))
ann.add(tf.keras.layers.Dense(units=256,activation="relu"))
ann.add(tf.keras.layers.Dense(units=3,activation="softmax"))
ann.compile(optimizer="adam", 
            loss="categorical_crossentropy", 
            metrics=['accuracy'])

In [ ]:
ann.fit(id_tr_emb, y_tr, batch_size=16, epochs = 100)

Epoch 1/100
232/232 [==============================] - 1s 3ms/step - loss: 0.9451 - accuracy: 0.6022
Epoch 2/100
232/232 [==============================] - 1s 3ms/step - loss: 0.8891 - accuracy: 0.6122
Epoch 3/100
232/232 [==============================] - 1s 2ms/step - loss: 0.7405 - accuracy: 0.6977
Epoch 4/100
232/232 [==============================] - 1s 2ms/step - loss: 0.7110 - accuracy: 0.7131
Epoch 5/100
232/232 [==============================] - 1s 3ms/step - loss: 0.6708 - accuracy: 0.7255
Epoch 6/100
232/232 [==============================] - 1s 2ms/step - loss: 0.6539 - accuracy: 0.7350
Epoch 7/100
232/232 [==============================] - 1s 2ms/step - loss: 0.6505 - accuracy: 0.7363
Epoch 8/100
232/232 [==============================] - 1s 2ms/step - loss: 0.6360 - accuracy: 0.7388
Epoch 9/100
232/232 [==============================] - 1s 2ms/step - loss: 0.6409 - accuracy: 0.7334
Epoch 10/100
232/232 [==============================] - 1s 2ms/step - loss: 0.6271 - accura